## HuggingFaceEmbeddings

1. HuggingFaceEmbeddings 란?**

`HuggingFaceEmbeddings`는 **Hugging Face에서 제공하는 사전 학습된 언어 모델을 사용하여 텍스트를 벡터(Embedding)로 변환하는 도구**입니다. 주로 **자연어 검색, 문서 유사도 분석, 의미 기반 검색** 등의 작업에 활용됩니다.

2. 주요 기능 및 역할**

- **텍스트를 벡터로 변환** → 기계가 이해할 수 있는 수치적 표현 생성
- **문서 검색 및 유사도 분석** → 벡터 간 거리를 계산하여 의미적 유사성 판단
- **Hugging Face 모델 사용** → `sentence-transformers` 기반 다양한 모델 지원
- **로컬 또는 GPU에서 실행 가능** → OpenAI API 없이도 실행 가능

3. **다양한 모델 비교**

- all-MiniLM-L6-v2: 소형, 빠른 속도 (384차원)
- all-mpnet-base-v2: 중형, 균형잡힌 성능 (768차원)
- paraphrase-multilingual-MiniLM-L12-v2: 다국어 지원

4. **한국어 처리 분석**

- 한국어 텍스트별 벡터 특성
- 벡터 크기, 평균, 표준편차 분석

In [2]:
from langchain_huggingface import HuggingFaceEmbeddings
import numpy as np
import time

In [5]:
# ==========================================
# 1. 기본 모델 로딩 및 임베딩
# ==========================================

print("\n1. 기본 모델 로딩 및 임베딩")
print("-" * 40)

# Hugging Face 임베딩 모델 사용 (소형 모델)
print("모델 로딩 중... (최초 실행시 다운로드 시간 소요)")
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
print("모델 로딩 완료")

# 모델 정보 확인
print(f"모델명: {embeddings.model_name}")

# 단일 문장 임베딩
query_text = "자연어 처리는 기계가 인간 언어를 이해하는 기술입니다."
print(f"\n쿼리 텍스트: {query_text}")

query_embedding = embeddings.embed_query(query_text)
print(f"쿼리 임베딩 차원: {len(query_embedding)}")
print(f"쿼리 임베딩 타입: {type(query_embedding)}")
print(f"벡터 샘플 (처음 10개): {query_embedding[:10]}")

# 여러 문장 임베딩
texts = [
    "인공지능은 다양한 산업에서 활용됩니다.",
    "자연어 처리는 언어를 분석하는 기술입니다.",
    "딥러닝을 활용한 모델이 성능이 좋습니다.",
    "머신러닝과 데이터 사이언스는 밀접한 관련이 있습니다."
]

print(f"\n문서 텍스트 ({len(texts)}개):")
for i, text in enumerate(texts, 1):
    print(f"  {i}. {text}")

doc_embeddings = embeddings.embed_documents(texts)
print(f"\n생성된 문서 벡터 개수: {len(doc_embeddings)}")
print(f"각 벡터 차원: {len(doc_embeddings[0])}")

# ==========================================
# 2. 다양한 HuggingFace 모델 비교
# ==========================================

print("\n" + "="*60)
print("2. 다양한 HuggingFace 모델 비교")
print("="*60)

# 여러 모델 설정 (크기와 성능이 다름)
model_configs = [
    {
        "name": "sentence-transformers/all-MiniLM-L6-v2",
        "description": "소형 모델, 빠른 속도",
        "dimensions": 384
    },
    {
        "name": "sentence-transformers/all-mpnet-base-v2", 
        "description": "중형 모델, 균형잡힌 성능",
        "dimensions": 768
    },
    {
        "name": "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
        "description": "다국어 지원 모델",
        "dimensions": 384
    }
]

test_text = "HuggingFace 모델 성능 비교 테스트"

for config in model_configs:
    print(f"\n모델: {config['name']}")
    print(f"설명: {config['description']}")
    print(f"예상 차원: {config['dimensions']}")
    
    try:
        # 모델 로딩 시간 측정
        start_time = time.time()
        model_embeddings = HuggingFaceEmbeddings(model_name=config['name'])
        load_time = time.time() - start_time
        
        # 임베딩 생성 시간 측정
        start_time = time.time()
        result = model_embeddings.embed_query(test_text)
        embed_time = time.time() - start_time
        
        print(f"로딩 시간: {load_time:.2f}초")
        print(f"임베딩 시간: {embed_time:.3f}초")
        print(f"실제 차원: {len(result)}")
        print(f"벡터 샘플: {result[:5]}")
        
    except Exception as e:
        print(f"오류: {e}")

# ==========================================
# 3. 모델 설정 옵션들
# ==========================================

print("\n" + "="*60)
print("3. 모델 설정 옵션들")
print("="*60)

# 다양한 설정으로 모델 생성
print("기본 설정:")
basic_embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

print("커스텀 설정:")
custom_embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={'device': 'cpu'},  # CPU 사용 명시
    encode_kwargs={'normalize_embeddings': True}  # L2 정규화 적용
)

# 설정별 결과 비교
test_sentence = "임베딩 정규화 테스트 문장입니다."

basic_result = basic_embeddings.embed_query(test_sentence)
custom_result = custom_embeddings.embed_query(test_sentence)

print(f"\n기본 설정 벡터 크기: {np.linalg.norm(basic_result):.6f}")
print(f"정규화 설정 벡터 크기: {np.linalg.norm(custom_result):.6f}")
print("정규화된 벡터는 크기가 1.0에 가까워집니다.")

# ==========================================
# 4. 벡터 유사도 분석
# ==========================================

print("\n" + "="*60)
print("4. 벡터 유사도 분석")
print("="*60)

def cosine_similarity(vec1, vec2):
    """코사인 유사도 계산"""
    vec1 = np.array(vec1)
    vec2 = np.array(vec2)
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

# 유사도 테스트용 문장들
similarity_texts = [
    "딥러닝은 인공지능의 핵심 기술입니다.",        # 기준
    "딥러닝은 AI의 중요한 기술입니다.",            # 매우 유사
    "머신러닝과 인공지능은 관련이 있습니다.",       # 유사
    "자연어 처리는 언어 분석 기술입니다.",         # 관련
    "오늘 날씨가 맑고 화창합니다."               # 무관
]

print("유사도 분석:")
vectors = embeddings.embed_documents(similarity_texts)
base_vector = vectors[0]  # 첫 번째를 기준으로

for i, (text, vector) in enumerate(zip(similarity_texts, vectors)):
    similarity = cosine_similarity(base_vector, vector)
    print(f"\n{i+1}. '{text}'")
    print(f"   유사도: {similarity:.4f}")
    
    if similarity > 0.8:
        status = "매우 유사"
    elif similarity > 0.6:
        status = "유사"
    elif similarity > 0.4:
        status = "관련"
    else:
        status = "무관"
    print(f"   상태: {status}")

# ==========================================
# 5. 성능 벤치마크 (HuggingFace vs OpenAI 시뮬레이션)
# ==========================================

print("\n" + "="*60)
print("5. 성능 벤치마크")
print("="*60)

# 테스트용 텍스트 생성
benchmark_texts = [
    f"벤치마크 테스트 문서 {i+1}번입니다. AI와 머신러닝에 대한 내용을 다룹니다."
    for i in range(20)
]

print(f"테스트 문서 수: {len(benchmark_texts)}개")

# HuggingFace 임베딩 성능 측정
print("\nHuggingFace 임베딩 성능:")
start_time = time.time()
hf_results = embeddings.embed_documents(benchmark_texts)
hf_time = time.time() - start_time

print(f"처리 시간: {hf_time:.3f}초")
print(f"문서당 평균 시간: {hf_time/len(benchmark_texts)*1000:.1f}ms")
print(f"벡터 차원: {len(hf_results[0])}")
print(f"메모리 사용량 (추정): {len(hf_results) * len(hf_results[0]) * 4 / 1024:.1f}KB")


1. 기본 모델 로딩 및 임베딩
----------------------------------------
모델 로딩 중... (최초 실행시 다운로드 시간 소요)


c:\Users\sogno\AppData\Local\pypoetry\Cache\virtualenvs\langchain-basic-farQSE-J-py3.12\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


KeyboardInterrupt: 